In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler


In [ ]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed
from tmqrfeed.manager import DataManager
from tmqrfeed.chains import OptionChainList, OptionChain
import pickle

In [ ]:
dm = DataManager()
feed = dm.datafeed
engine = feed.data_engine

In [ ]:
chain = feed.get_fut_chain("US.ES")
chain_values = chain.get_list(datetime(2012, 1, 1))

In [ ]:
chain_values    

In [ ]:
underlying = chain_values[-7][0]

In [ ]:
underlying

In [ ]:
chains_optcode = feed.data_engine.db_get_option_chains(underlying.ticker)

In [ ]:
with open('option_chain_list_es_optcode.pkl', 'wb') as fh:
    pickle.dump(chains_optcode, fh)

In [ ]:
chain_list = feed.get_option_chains(underlying)

In [ ]:
chain_list

In [ ]:
for chain_rec in chain_values:
    print(chain_rec[0])
    print(repr(feed.get_option_chains(chain_rec[0])))